### Crie as tabelas indicadas em ModeloBD.png


- 1 - Carregar os dados dos arquivos nas respectivas tabelas 
- 2 - Quantos estudantes obtiveram média acima de 83.5 ? 
- 3 - Faça uma consulta que retorne os cursos dados ordenados em "ordem" alfabética. 
- 4 - Faça uma consulta que retorne o nome de todas as pessoas cujo o último nome termina com "ez"
- 5 - Quantos estudantes precisaram de 3 tentativas para serem aprovados 
- 6 - Usando uma lógica condicional retorne : Id do aluno, nota, aprovado ou reprovado. Aprovado se nota >= 70.0 
  * Dica: Fazer conforme o exemplo ### Using Conditional Logic in a SELECT Statement ####  do notebook 
- 7 - Realizar uma consulta que retorne os nomes dos cursos cuja a média obtida (AVG) pelo alunos foi maior ou igual 75.0  
- 8 - Realizar uma consulta que retorne os email dos alunos que cursaram 3 vezes um mesmo curso 
- 10 -  (DESAFIO) Realizar uma consulta que retorne os nomes dos professores mais carrascos - isto é - aqueles cujo o número de alunos reprovados (média menor que 70.0) foi maior do que de
 alunos aprovados.

In [1]:
import sqlite3
from decouple import config

In [2]:
# Create Env vars
DATA_SQL_PATH = config("DATA_SQL_PATH")

#### Criando o banco de dados SQLITE

In [3]:
engine = sqlite3.connect(DATA_SQL_PATH+"dataChallenge.db")
# Create a cursor
cursor = engine.cursor()

#### 1 - Carregar os dados dos arquivos nas respectivas tabelas 

In [4]:
# Função para fazer o load dos arquivos
def load_data_fromFile_txt (file_path:str) :

    with open(file_path, mode="r") as file :
        data = file.readlines()

    data = [line.strip().split(",") for line in data]

    return data

* Carregando os dados dos txt em variáveis

In [5]:
# Load course data
courseData = load_data_fromFile_txt(DATA_SQL_PATH+"SampleCourseData.txt")
# Load Credit data
creditData = load_data_fromFile_txt(DATA_SQL_PATH+"SampleCreditData.txt")
# Load Person data
personData = load_data_fromFile_txt(DATA_SQL_PATH+"SamplePersonData.txt")
# Load Student data
studentData = load_data_fromFile_txt(DATA_SQL_PATH+"SampleStudentData.txt")

#### Pre Process Data

In [6]:
# Pre process data COURSE
for line in courseData :
    line[0] = int(line[0])
# Pre process data CREDIT
for line in creditData :
    line[0] = int(line[0])
    line[1] = int(line[1])
    line[2] = float(line[2])
    line[3] = int(line[3])
# Pre process data PERSON
for line in personData :
    line[0] = int(line[0])
# Pre process data PERSON
for line in studentData :
    line[0] = int(line[0])
    line[1] = int(line[1])

* Criando os comandos de criação das tabelas

In [7]:
# Query to create table COURSE
courseDataQuery = '''
    CREATE TABLE IF NOT EXISTS course (
    courseId INTEGER PRIMARY KEY,
    name TEXT,
    teacher TEXT
    );'''
# Query to create table CREDIT
creditDataQuery = '''
    CREATE TABLE IF NOT EXISTS credit (
    courseId INTEGER,
    studentId INTEGER,
    grade REAL,
    attempt INTEGER,
    FOREIGN KEY (courseId) REFERENCES course (courseId)
    FOREIGN KEY (studentId) REFERENCES student (studentId)
    );'''
# Query to create table PERSON
personDataQuery = '''
    CREATE TABLE IF NOT EXISTS person (
    personId INTEGER PRIMARY KEY,
    firstName TEXT,
    middleInitial TEXT,
    lastName TEXT,
    dateOfBirth DATE
    );'''
# Query to create table STUDENT
studentDataQuery = '''
    CREATE TABLE IF NOT EXISTS student (
    studentId INTEGER PRIMARY KEY,
    personId INTEGER,
    email TEXT,
    FOREIGN KEY (personId) REFERENCES person (personId)
    );'''

* Execute creation of COURSE table

In [8]:
cursor.execute(courseDataQuery)
cursor.execute("PRAGMA table_info(course)").fetchall()

[(0, 'courseId', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'teacher', 'TEXT', 0, None, 0)]

* Execute creation of CREDIT table

In [9]:
cursor.execute(creditDataQuery)
cursor.execute("PRAGMA table_info(credit)").fetchall()

[(0, 'courseId', 'INTEGER', 0, None, 0),
 (1, 'studentId', 'INTEGER', 0, None, 0),
 (2, 'grade', 'REAL', 0, None, 0),
 (3, 'attempt', 'INTEGER', 0, None, 0)]

* Execute creation of PERSON table

In [10]:
cursor.execute(personDataQuery)
cursor.execute("PRAGMA table_info(person)").fetchall()

[(0, 'personId', 'INTEGER', 0, None, 1),
 (1, 'firstName', 'TEXT', 0, None, 0),
 (2, 'middleInitial', 'TEXT', 0, None, 0),
 (3, 'lastName', 'TEXT', 0, None, 0),
 (4, 'dateOfBirth', 'DATE', 0, None, 0)]

* Execute creation of STUDENT table

In [11]:
cursor.execute(studentDataQuery)
cursor.execute("PRAGMA table_info(student)").fetchall()

[(0, 'studentId', 'INTEGER', 0, None, 1),
 (1, 'personId', 'INTEGER', 0, None, 0),
 (2, 'email', 'TEXT', 0, None, 0)]

#### 1.1 - Popular as tabelas

In [12]:
def populateTables (data:list, table:str, cursor:sqlite3.Cursor) : 
    

    table_schema = cursor.execute(f"PRAGMA table_info({table})").fetchall()
    table_schema = tuple([str(elem[1]) for elem in table_schema])
    print(f"Columns : {table_schema}")
    for line in data :
        line = tuple(line)
        
        cursor.execute(
            f'''
                INSERT INTO {table} {table_schema} VALUES {line};
            '''
        )
    
    verify_add = cursor.execute(
        f"SELECT * FROM {table}"
    ).fetchall()

    return verify_add

In [13]:
# Populate COURSE table
populateTables(data=courseData, table='course', cursor=cursor)
# Populate CREDIT table
populateTables(data=creditData, table='credit', cursor=cursor)
# Populate PERSON table
populateTables(data=personData, table='person', cursor=cursor)
# Populate STUDENT table
populateTables(data=studentData, table='student', cursor=cursor)

Columns : ('courseId', 'name', 'teacher')
Columns : ('courseId', 'studentId', 'grade', 'attempt')
Columns : ('personId', 'firstName', 'middleInitial', 'lastName', 'dateOfBirth')
Columns : ('studentId', 'personId', 'email')


[(1, 1, 'AdeleRosales@university.com'),
 (2, 2, 'JustusGomez@university.com'),
 (3, 3, 'IsiahWarner@university.com'),
 (4, 4, 'EricaCampos@university.com'),
 (5, 5, 'DesmondValdez@university.com'),
 (6, 6, 'ArnavPowers@university.com'),
 (7, 7, 'AliBowen@university.com'),
 (8, 8, 'MalayaBurgess@university.com'),
 (9, 9, 'HaleighHopper@university.com'),
 (10, 10, 'CoreyMcclure@university.com'),
 (11, 11, 'JacobDotson@university.com'),
 (12, 12, 'AdenPetty@university.com'),
 (13, 13, 'TheodoreAcosta@university.com'),
 (14, 14, 'HarleyManning@university.com'),
 (15, 15, 'KarmaGuerrero@university.com'),
 (16, 16, 'LegendOlson@university.com'),
 (17, 17, 'IssacLee@university.com'),
 (18, 18, 'MaggieYork@university.com'),
 (19, 19, 'AmareHernandez@university.com'),
 (20, 20, 'LilahAbbott@university.com'),
 (21, 21, 'EugeneMathis@university.com'),
 (22, 22, 'MiyaWarren@university.com'),
 (23, 23, 'MuhammadScott@university.com'),
 (24, 24, 'ErikMcmillan@university.com'),
 (25, 25, 'ValentinoCh

#### 2 - Quantos estudantes obtiveram média acima de 83.5 ? 

In [14]:
cursor.execute(
    '''
    SELECT COUNT(studentID)
    FROM credit
    WHERE grade > 83.5
    '''
).fetchall()

[(197,)]

#### 3 - Faça uma consulta que retorne os cursos dados ordenados em "ordem" alfabética. 

In [21]:
cursor.execute(
    '''
    SELECT DISTINCT(name)
    FROM course
    ORDER BY name ASC
    '''
).fetchall()

[('Arts Fundraising',),
 ('Calculus',),
 ('Circuit Analysis',),
 ('Elementary French ',),
 ('Financial Accounting',),
 ('Food and Culture',),
 ('Foundations of Education',),
 ('Fundamentals of Marketing ',),
 ('Greek Civilization',),
 ('Human Ecology',),
 ('Introduction to Affective Computing',),
 ('Introduction to Law ',),
 ('Introduction to Philosophy',),
 ('Introduction to Professional Writing',),
 ('Introduction to Programming',),
 ('Literature and Culture',),
 ('Network Optimization',),
 ('World History',)]

#### 4 - Faça uma consulta que retorne o nome de todas as pessoas cujo o último nome termina com "ez"

In [20]:
cursor.execute(
    '''
    SELECT firstName, lastName
    FROM person
    WHERE lastname LIKE '%ez'
    '''
).fetchall()

[('Justus', 'Gomez'),
 ('Desmond', 'Valdez'),
 ('Amare', 'Hernandez'),
 ('Jaida', 'Martinez'),
 ('Aaron', 'Perez'),
 ('Ava', 'Velez'),
 ('Giada', 'Fernandez'),
 ('Ruben', 'Rodriquez'),
 ('Seth', 'Marquez'),
 ('Alexis', 'Velez'),
 ('Nicolas', 'Gutierrez'),
 ('Britney', 'Sanchez'),
 ('Ayana', 'Vasquez'),
 ('Angeline', 'Velasquez'),
 ('Delilah', 'Jimenez'),
 ('Demarcus', 'Martinez'),
 ('Zane', 'Suarez'),
 ('Yandel', 'Valdez'),
 ('Lillian', 'Martinez'),
 ('Xavier', 'Lopez'),
 ('Adelaide', 'Jimenez'),
 ('Waylon', 'Alvarez'),
 ('Kaia', 'Sanchez'),
 ('Aliza', 'Vazquez'),
 ('Jorden', 'Cortez'),
 ('Isiah', 'Rodriguez')]